# 03 Tree Models and Feature Importance

Tree classifiers and feature importance vs permutation.


## Table of Contents
- [Fit tree model](#fit-tree-model)
- [Compare metrics](#compare-metrics)
- [Interpret importance](#interpret-importance)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## Prerequisites (Quick Self-Check)
- Completed Part 02 (regression basics) or equivalent.
- Comfort interpreting probabilities and trade-offs (false positives vs false negatives).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/03_classification/03_tree_models_and_importance.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/03_classification/03_tree_models_and_importance.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Compare a tree-based classifier to logistic regression and interpret feature importance.

Trees can capture non-linearities and interactions, but are easier to overfit.



## Primer: sklearn pipelines (how to avoid preprocessing leakage)

Pipelines prevent a common mistake: fitting preprocessing (scaling, imputation) using information from the test set.

### Why pipelines exist (in one sentence)

> A `Pipeline` ensures that transformations are fit on training data only, then applied to test data.

### The key APIs

- `fit(X, y)`: learn parameters (scaler mean/std, model weights) from training.
- `transform(X)`: apply learned transform to new data.
- `fit_transform(X, y)`: convenience for training data only.

### Minimal pattern (classification)

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
  ("scaler", StandardScaler()),
  ("model", LogisticRegression(max_iter=5000)),
])

# clf.fit(X_train, y_train)
# y_prob = clf.predict_proba(X_test)[:, 1]
```

**Expected output / sanity check**
- you never call `scaler.fit` on the full dataset
- you split by time first, then fit the pipeline on train

### Mini demo: the leakage you’re avoiding (toy)

```python
import numpy as np
from sklearn.preprocessing import StandardScaler

rng = np.random.default_rng(0)
X_train = rng.normal(loc=0.0, scale=1.0, size=(100, 1))
X_test  = rng.normal(loc=2.0, scale=1.0, size=(25, 1))

# WRONG: fit on train+test (leaks the future)
sc_wrong = StandardScaler().fit(np.vstack([X_train, X_test]))
X_test_wrong = sc_wrong.transform(X_test)

# RIGHT: fit on train only
sc_right = StandardScaler().fit(X_train)
X_test_right = sc_right.transform(X_test)

print("test mean after wrong scaling:", float(X_test_wrong.mean()))
print("test mean after right scaling:", float(X_test_right.mean()))
```

### Common pitfalls

- Splitting after preprocessing (leakage).
- Using random splits on time-indexed data (temporal leakage).
- Forgetting `ColumnTransformer` for mixed numeric/categorical columns (if needed).


<a id="fit-tree-model"></a>
## Fit tree model

### Goal
Fit a tree-based classifier on the recession prediction task.



### Your Turn (1): Load data and split


In [ ]:
import pandas as pd
from src.evaluation import time_train_test_split_index

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_col = 'target_recession_next_q'
x_cols = ['T10Y2Y_lag1', 'UNRATE_lag1', 'FEDFUNDS_lag1', 'INDPRO_lag1', 'RSAFS_lag1']
df_m = df[[y_col] + x_cols].dropna().copy()

split = time_train_test_split_index(len(df_m), test_size=0.2)
train = df_m.iloc[split.train_slice]
test = df_m.iloc[split.test_slice]

X_train = train[x_cols]
y_train = train[y_col].astype(int)
X_test = test[x_cols]
y_test = test[y_col].astype(int)



### Your Turn (2): Fit a RandomForestClassifier


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# TODO: Fit a simple random forest.
# Start small to avoid overfitting.
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=3,
    random_state=0,
)

rf.fit(X_train, y_train)
p_test = rf.predict_proba(X_test)[:, 1]
p_test[:5]



<a id="compare-metrics"></a>
## Compare metrics

### Goal
Compare tree performance to a logistic baseline.



### Your Turn: Evaluate and compare


In [ ]:
from src.evaluation import classification_metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Logistic baseline
logit = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=5000)),
])
logit.fit(X_train, y_train)
p_logit = logit.predict_proba(X_test)[:, 1]

m_rf = classification_metrics(y_test.to_numpy(), p_test, threshold=0.5)
m_logit = classification_metrics(y_test.to_numpy(), p_logit, threshold=0.5)

{'random_forest': m_rf, 'logistic': m_logit}



<a id="interpret-importance"></a>
## Interpret importance

### Goal
Compare built-in feature importance to permutation importance.

Why they can disagree:
- built-in importance can be biased toward high-cardinality/noisy features
- permutation importance measures impact on a chosen metric on a chosen dataset



### Your Turn (1): Built-in feature importances


In [ ]:
import pandas as pd

imp = pd.Series(rf.feature_importances_, index=x_cols).sort_values(ascending=False)
imp



### Your Turn (2): Permutation importance


In [ ]:
import pandas as pd
from sklearn.inspection import permutation_importance

# TODO: Compute permutation importance using ROC-AUC or average_precision.
r = permutation_importance(
    rf,
    X_test,
    y_test,
    n_repeats=30,
    random_state=0,
    scoring='roc_auc',
)

perm = pd.Series(r.importances_mean, index=x_cols).sort_values(ascending=False)
perm



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Fit tree model</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_tree_models_and_importance — Fit tree model
from sklearn.ensemble import RandomForestClassifier
from src import evaluation

rf = RandomForestClassifier(n_estimators=300, random_state=0)
rf.fit(X_tr, y_tr)
p_rf = rf.predict_proba(X_te)[:,1]
evaluation.classification_metrics(y_te.to_numpy(), p_rf)
```

</details>

<details><summary>Solution: Compare metrics</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_tree_models_and_importance — Compare metrics
# Compare to logistic metrics from earlier.
```

</details>

<details><summary>Solution: Interpret importance</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_tree_models_and_importance — Interpret importance
import pandas as pd
from sklearn.inspection import permutation_importance

imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
imp.head(10)

perm = permutation_importance(rf, X_te, y_te, n_repeats=10, random_state=0)
perm_imp = pd.Series(perm.importances_mean, index=X.columns).sort_values(ascending=False)
perm_imp.head(10)
```

</details>

